In [4]:
from importlib.metadata import version

pkgs = ["matplotlib",
        "numpy",
        "tiktoken",
        "torch",
        "tensorflow" #For OpenAI's pretrained weights
       ]

for p in pkgs:
    print(f"{p} version: {version(p)}")


matplotlib version: 3.10.5
numpy version: 2.2.6
tiktoken version: 0.11.0
torch version: 2.8.0
tensorflow version: 2.20.0
